# Complete Attendance Processing Workflow

**Complete working workflow from 5 input files to final Excel report (Going to be final)**

### Steps:
1. Load 5 input files (GIMT, GIPS, ADMIN, LEAVE, EXEMPTED)
2. Pad column names (normalize dates)
3. Detect and remove holidays (HOLIDAY_LIST + user extras)
4. Calculate working days
5. Merge using merge_files_staffs
6. Consolidate and rename columns
7. Generate final Excel report

**HOLIDAY_LIST preserved - NEVER DELETE**  
**Full pipeline exactly as working in hr_attendance.py**


In [1]:
# Cell 1: Complete Imports and Setup
import pandas as pd
import numpy as np
import streamlit as st  # For accessing secrets
from utility_attendance import split_file, merge_files_staffs, detect_holidays_staffs, calculate_working_days, pad_month_in_columns
from utility import connect_gsheet, get_dataframe
from datetime import datetime

In [2]:
import warnings
warnings.filterwarnings("ignore")

### 1. Holiday Set up

In [3]:
# DO NOT DELETE THIS LIST
HOLIDAY_LIST = ['29-sep-2025','30-sep-2025','01-oct-2025','02-oct-2025','03-oct-2025',
                '06-oct-2025','18-oct-2025','20-oct-2025','21-oct-2025','05-nov-2025','25-dec-2025']

# User extra holidays
misc_holidays = "23-sep-2025"  # jubin garg dead
misc_working_days = ""

# Extend HOLIDAY_LIST (hr_attendance.py lines 148-152)
misc_holidays_list = [h.strip() for h in misc_holidays.split(',') if h.strip()] if misc_holidays else []
all_holidays = HOLIDAY_LIST.copy()
if misc_holidays_list:
    all_holidays.extend(misc_holidays_list)
    
print(f"📅 Total holidays: {len(all_holidays)}")
for h in all_holidays[:5]:
    print(f"  - {h}")
if len(all_holidays) > 5:
    print(f"  ... and {len(all_holidays)-5} more")


📅 Total holidays: 12
  - 29-sep-2025
  - 30-sep-2025
  - 01-oct-2025
  - 02-oct-2025
  - 03-oct-2025
  ... and 7 more


### 2. Loading files

In [4]:
#  Load ALL 5 Input Files
print("📂 Loading GIMT, GIPS, ADMIN, LEAVE, EXEMPTED...")

# Load and split files
# 
df_gimt = pd.read_excel('./data/GIMT_MonthlyAttendanceSummaryReport (29).xlsx')
df_admin = pd.read_excel('./data/Admin_MonthlyAttendanceSummaryReport(48).xlsx')
df_gips = pd.read_excel('./data/GIPS_MonthlyAttendanceSummaryReport(46).xlsx')
df_leave_erp = pd.read_csv('./data/Leave Report Oct25.csv',skiprows=6, encoding='windows-1252')
exempted_file = pd.ExcelFile('./data/template_exempted.xlsx')


📂 Loading GIMT, GIPS, ADMIN, LEAVE, EXEMPTED...


In [5]:
### Reading EMP data from ERP
#df_emp = pd.read_csv('./data/emp_master_data.csv', skiprows=6, encoding='windows-1252') 
df_emp = pd.read_csv('./data/2015_10_27_employee_list.csv', skiprows=6, encoding='windows-1252')  

In [6]:
#df_emp[df_emp['Employee ID']=='GCU020003']

In [7]:
#df_emp.

In [8]:
df_gimt_all, df_gimt_in, df_gimt_out = split_file(df_gimt)
df_gips_all, df_gips_in, df_gips_out = split_file(df_gips)
df_admin_all, df_admin_in, df_admin_out = split_file(df_admin)

#### 2.1 Padding columns names

In [9]:
df_gimt_in = pad_month_in_columns(df_gimt_in, 'clock_in')
df_gips_in = pad_month_in_columns(df_gips_in, 'clock_in')
df_admin_in = pad_month_in_columns(df_admin_in, 'clock_in')
df_gimt_all = pad_month_in_columns(df_gimt_all, 'clock_in')
df_gips_all = pad_month_in_columns(df_gips_all, 'clock_in')
df_admin_all = pad_month_in_columns(df_admin_all, 'clock_in')

df_gimt_out = pad_month_in_columns(df_gimt_out, 'clock_out')
df_gips_out = pad_month_in_columns(df_gips_out, 'clock_out')
df_admin_out = pad_month_in_columns(df_admin_out, 'clock_out')
df_gimt_all = pad_month_in_columns(df_gimt_all, 'clock_out')
df_gips_all = pad_month_in_columns(df_gips_all, 'clock_out')
df_admin_all = pad_month_in_columns(df_admin_all, 'clock_out')

In [10]:
# Pad month in columns (normalize clock_in_9_6 to clock_in_09_06)
#df_gimt_in = pad_month_in_columns(df_gimt_in, 'clock_in')
#df_gimt_out = pad_month_in_columns(df_gimt_out, 'clock_out')
#df_gips_in = pad_month_in_columns(df_gips_in, 'clock_in')
#df_gips_out = pad_month_in_columns(df_gips_out, 'clock_out')
#df_admin_in = pad_month_in_columns(df_admin_in, 'clock_in')
#df_admin_out = pad_month_in_columns(df_admin_out, 'clock_out')

In [11]:
# holidays - 7
# total 10+25 days
#'20-oct-2025','21-oct-2025',4,5,12,18,19

In [12]:
df_gimt_in.columns

Index(['Emp Id', 'Names', 'Present', 'clock_in_09_21', 'clock_in_09_22',
       'clock_in_09_23', 'clock_in_09_24', 'clock_in_09_25', 'clock_in_09_26',
       'clock_in_09_27', 'clock_in_09_28', 'clock_in_09_29', 'clock_in_09_30',
       'clock_in_10_01', 'clock_in_10_02', 'clock_in_10_03', 'clock_in_10_04',
       'clock_in_10_05', 'clock_in_10_06', 'clock_in_10_07', 'clock_in_10_08',
       'clock_in_10_09', 'clock_in_10_10', 'clock_in_10_11', 'clock_in_10_12',
       'clock_in_10_13', 'clock_in_10_14', 'clock_in_10_15', 'clock_in_10_16',
       'clock_in_10_17', 'clock_in_10_18', 'clock_in_10_19', 'clock_in_10_20',
       'clock_in_10_21', 'clock_in_10_22', 'clock_in_10_23', 'clock_in_10_24',
       'clock_in_10_25'],
      dtype='object')

### 3. Detect Holidays

In [13]:
# Detect holidays (hr_attendance.py lines 162-163)
holidays = detect_holidays_staffs(df_gimt_in, year=2025, misc_holidays=all_holidays, misc_working_days=misc_working_days, verbose=False)
print(f"🗓️ Detected {len(holidays)} holidays")

# Remove holidays (hr_attendance.py lines 167-168)
cols_to_delete_in = holidays
cols_to_delete_out = [c.replace('clock_in', 'clock_out') for c in holidays]

df_gimt_in = df_gimt_in.drop(columns=cols_to_delete_in, axis=1, errors='ignore')
df_gimt_out = df_gimt_out.drop(columns=cols_to_delete_out, axis=1, errors='ignore')
df_gips_in = df_gips_in.drop(columns=cols_to_delete_in, axis=1, errors='ignore')
df_gips_out = df_gips_out.drop(columns=cols_to_delete_out, axis=1, errors='ignore')
df_admin_in = df_admin_in.drop(columns=cols_to_delete_in, axis=1, errors='ignore')
df_admin_out = df_admin_out.drop(columns=cols_to_delete_out, axis=1, errors='ignore')

print(f"✅ Removed holidays from all dataframes")
print(f"📊 Final shapes: GIMT={df_gimt_in.shape}, GIPS={df_gips_in.shape}, ADMIN={df_admin_in.shape}")


🗓️ Detected 16 holidays
✅ Removed holidays from all dataframes
📊 Final shapes: GIMT=(120, 22), GIPS=(41, 22), ADMIN=(145, 22)


In [14]:
final_working_days = df_gimt_in.columns
final_working_days

Index(['Emp Id', 'Names', 'Present', 'clock_in_09_22', 'clock_in_09_24',
       'clock_in_09_25', 'clock_in_09_26', 'clock_in_09_27', 'clock_in_10_07',
       'clock_in_10_08', 'clock_in_10_09', 'clock_in_10_10', 'clock_in_10_11',
       'clock_in_10_13', 'clock_in_10_14', 'clock_in_10_15', 'clock_in_10_16',
       'clock_in_10_17', 'clock_in_10_22', 'clock_in_10_23', 'clock_in_10_24',
       'clock_in_10_25'],
      dtype='object')

#### 3.1 Calculate Working Days

In [15]:
# Calculate from actual data AFTER removing holidays (hr_attendance.py lines 228-229)
working_days_list = calculate_working_days(df_gimt_in)
no_working_days = len(df_gimt_in.columns) - 3
print(f"✅ Working days: {no_working_days}")


✅ Working days: 19


In [16]:
emp_df = df_emp[['Employee ID','Name','Designation','Department']]
emp_df = emp_df.rename(columns={'Employee ID':'Emp Id'})

# Reset index to avoid issues
emp_df.reset_index(drop=True, inplace=True)

In [17]:
# Cell 5: Merge files using merge_files_staffs (like working notebook)
print("🔄 Merging files with merge_files_staffs...")

# Use merge_files_staffs for all staff (hr_attendance.py lines 174-176)
# Note: merge_files_staffs expects emp_df as DataFrame, not indexed
df_gimt_merged = merge_files_staffs(df_gimt_in, df_gimt_out, emp_df.copy(), no_working_days, all_holidays, misc_working_days)
df_gips_merged = merge_files_staffs(df_gips_in, df_gips_out, emp_df.copy(), no_working_days, all_holidays, misc_working_days)
df_admin_merged = merge_files_staffs(df_admin_in, df_admin_out, emp_df.copy(), no_working_days, all_holidays, misc_working_days)

print(f"✅ GIMT merged: {df_gimt_merged.shape}")
print(f"✅ GIPS merged: {df_gips_merged.shape}")
print(f"✅ ADMIN merged: {df_admin_merged.shape}")

🔄 Merging files with merge_files_staffs...
✅ GIMT merged: (120, 18)
✅ GIPS merged: (41, 18)
✅ ADMIN merged: (145, 18)


In [59]:
#df_gimt_merged.head()

In [18]:
# Check for GCU010013 (Raisa Choudhury)
if 'GCU010013' in df_gimt_merged['Emp Id'].values:
    debug_merged = df_gimt_merged[df_gimt_merged['Emp Id'] == 'GCU010013']
    print("\n🔍 DEBUG GCU010013:")
    print(debug_merged[['Emp Id', 'Name', 'Working Days', 'Present', 'Absent']].to_string())

print("\n✅ All files merged!")


🔍 DEBUG GCU010013:
      Emp Id       Name  Working Days  Present  Absent
0  GCU010013  Ms. Raisa            19     18.0     1.0

✅ All files merged!


### 4. Consolidate and Merge with Employee Data

In [63]:
df_gimt_all.shape

(120, 72)

In [64]:
df_gips_all.shape

(41, 72)

In [65]:
df_fac_all_rp = pd.concat([df_gimt_all, df_gips_all], ignore_index=True)
df_fac_all_rp.head()

,Emp Id,Names,clock_in_09_21,clock_in_09_22,clock_in_09_23,clock_in_09_24,clock_in_09_25,clock_in_09_26,clock_in_09_27,clock_in_09_28,...,clock_out_10_16,clock_out_10_17,clock_out_10_18,clock_out_10_19,clock_out_10_20,clock_out_10_21,clock_out_10_22,clock_out_10_23,clock_out_10_24,clock_out_10_25
0,GCU010013,Ms. Raisa,0,0,0,08:41:00,08:38:00,08:40:00,08:41:00,0,...,16:20:00,16:49:00,0,0,0,0,16:26:00,18:29:00,17:32:00,18:06:00
1,GCU010017,Ms. Ananya,0,08:25:00,0,08:42:00,08:43:00,08:37:00,08:33:00,0,...,16:20:00,16:47:00,0,0,0,0,16:21:00,16:22:00,17:32:00,18:05:00
2,GCU020001,Dr. Lipi,0,08:31:00,0,08:34:00,08:36:00,08:38:00,08:44:00,0,...,16:26:00,16:47:00,0,0,0,0,16:20:00,16:25:00,17:41:00,18:07:00
3,GCU020002,Dr. Swarnali,0,08:41:00,0,08:43:00,08:43:00,08:43:00,08:42:00,0,...,16:26:00,16:53:00,0,0,0,0,17:12:00,16:34:00,17:43:00,18:14:00
4,GCU020003,Dr. Kajal,0,08:28:00,0,08:35:00,08:29:00,08:31:00,08:22:00,0,...,17:04:00,16:48:00,0,0,0,0,16:42:00,16:23:00,17:32:00,18:11:00


In [69]:
df_admin_all_rp = df_admin_all.copy()
df_admin_all_rp.shape

(145, 72)

In [19]:
# Combine GIMT and GIPS for faculties
df_fac_detail = pd.concat([df_gimt_all, df_gips_all], ignore_index=True)
df_fac_conso = pd.concat([df_gimt_merged, df_gips_merged], ignore_index=True)

df_admin_detail = df_admin_all.copy()
df_admin_conso = df_admin_merged.copy()

print(f"✅ Faculty consolidated: {df_fac_conso.shape}")
print(f"✅ Admin consolidated: {df_admin_conso.shape}")

# Rename columns for report format
col_to_rename_staffs = {
    'AM_abs':'actual_AM_abs',
    'PM_abs':'actual_PM_abs',
    'days_abs':'actual_days_abs'
}
df_fac_conso.rename(columns=col_to_rename_staffs, inplace=True)
df_admin_conso.rename(columns=col_to_rename_staffs, inplace=True)


✅ Faculty consolidated: (161, 18)
✅ Admin consolidated: (145, 18)


In [62]:
df_fac_detail.head()

,Emp Id,Names,clock_in_09_22,clock_in_09_24,clock_in_09_25,clock_in_09_26,clock_in_09_27,clock_in_10_07,clock_in_10_08,clock_in_10_09,...,clock_in_10_11,clock_in_10_13,clock_in_10_14,clock_in_10_15,clock_in_10_16,clock_in_10_17,clock_in_10_22,clock_in_10_23,clock_in_10_24,clock_in_10_25
0,GCU010013,Ms. Raisa,0,08:41:00,08:38:00,08:40:00,08:41:00,08:39:00,08:39:00,08:40:00,...,08:38:00,08:41:00,08:39:00,08:38:00,09:41:00,08:39:00,08:38:00,09:33:00,08:37:00,08:38:00
1,GCU010017,Ms. Ananya,08:25:00,08:42:00,08:43:00,08:37:00,08:33:00,08:32:00,08:36:00,08:35:00,...,08:29:00,08:29:00,08:33:00,08:33:00,08:32:00,08:44:00,08:26:00,08:28:00,08:33:00,08:28:00
2,GCU020001,Dr. Lipi,08:31:00,08:34:00,08:36:00,08:38:00,08:44:00,08:32:00,08:34:00,08:33:00,...,08:37:00,08:40:00,08:35:00,08:35:00,08:34:00,08:30:00,08:31:00,08:31:00,08:31:00,08:43:00
3,GCU020002,Dr. Swarnali,08:41:00,08:43:00,08:43:00,08:43:00,08:42:00,08:36:00,08:42:00,08:36:00,...,08:41:00,08:40:00,08:42:00,08:33:00,08:40:00,08:35:00,08:40:00,08:35:00,08:38:00,08:36:00
4,GCU020003,Dr. Kajal,08:28:00,08:35:00,08:29:00,08:31:00,08:22:00,08:29:00,08:22:00,08:23:00,...,08:28:00,08:22:00,08:27:00,08:22:00,08:26:00,08:38:00,08:26:00,08:27:00,08:32:00,08:37:00


#### 4.1 Selecting only the working days columns

In [20]:
common_cols_fac = [col for col in final_working_days if col in df_fac_detail.columns]
common_cols_admin = [col for col in final_working_days if col in df_admin_detail.columns]

df_fac_detail = df_fac_detail[common_cols_fac]
df_admin_detail = df_admin_detail[common_cols_admin]


In [21]:
# Merge with employee master data
df_fac_conso_ID = pd.merge(df_fac_conso, emp_df, how='left', on='Emp Id', suffixes=('', '_from_emp'))
df_admin_conso_ID = pd.merge(df_admin_conso, emp_df, how='left', on='Emp Id', suffixes=('', '_from_emp'))

# Drop duplicate columns from emp_df
cols_to_drop_emp = [col for col in df_fac_conso_ID.columns if col.endswith('_from_emp')]
cols_to_drop_emp = [col for col in df_admin_conso_ID.columns if col.endswith('_from_emp')]

df_fac_conso_ID = df_fac_conso_ID.drop(columns=cols_to_drop_emp, errors='ignore')
df_admin_conso_ID = df_admin_conso_ID.drop(columns=cols_to_drop_emp, errors='ignore')

print(f"✅ Merged with employee data")
print(f"   Faculty: {df_fac_conso_ID.shape}, Admin: {df_admin_conso_ID.shape}")

✅ Merged with employee data
   Faculty: (161, 18), Admin: (145, 18)


In [22]:
df_fac_conso_ID.head()

,Emp Id,Name,Designation,Department,late_flags,early_flags,half_day_flags,actual_AM_abs,actual_PM_abs,actual_days_abs,No_of_AM_abs,No_of_PM_abs,actual_No_of_late,actual_half_day,actual_full_day,Working Days,Present,Absent
0,GCU010013,Ms. Raisa,-,Administration,"10_16, 10_23",,,,,09_22,0,0,2,0,1,19,18.0,1.0
1,GCU010017,Ms. Ananya,-,Administration,,,,,,,0,0,0,0,0,19,19.0,0.0
2,GCU020001,Dr. Lipi,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,,0,0,0,0,0,19,19.0,0.0
3,GCU020002,Dr. Swarnali,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,,0,0,0,0,0,19,19.0,0.0
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,,,,,,,0,0,0,0,0,19,19.0,0.0


### 5. Process EXEMPTED and LEAVE Data

In [23]:
# Process exempted leaves
from utility_attendance import process_exempted_leaves
df_exempted = process_exempted_leaves(exempted_file)
df_exempted.rename(columns={'late_count':'exempt_late','half_day_count':'exempt_HD','full_day_count':'exempt_FD'}, inplace=True)
if 'Name' in df_exempted.columns:
    df_exempted.drop('Name',axis=1,inplace=True)

print(f"✅ EXEMPTED processed: {df_exempted.shape}")

✅ EXEMPTED processed: (179, 4)


In [24]:
# Process LEAVE ERP data  
from utility import preprocess_date
if 'From Date' in df_leave_erp.columns:
    df_leave_erp['From Date'] = df_leave_erp['From Date'].apply(preprocess_date)
    df_leave_erp['From Date'] = pd.to_datetime(df_leave_erp['From Date'], errors='coerce')
if 'To Date' in df_leave_erp.columns:
    df_leave_erp['To Date'] = df_leave_erp['To Date'].apply(preprocess_date)
    df_leave_erp['To Date'] = pd.to_datetime(df_leave_erp['To Date'], errors='coerce')

from utility_attendance import calculate_leave_summary_with_wd_leaves
df_leave_erp_summary = calculate_leave_summary_with_wd_leaves(df_leave_erp, working_days_list)
df_leave_erp_summary.fillna(0, inplace=True)

df_leave_erp_summary['Approved leaves (ERP)'] = df_leave_erp_summary['Total WD leaves'] + df_leave_erp_summary['Casual Leave']

cols_to_drop = [
    "Casual Leave", "Sick Leave", "Duty Leave", "Vacation Leave",
    "Maternity Leave", "Earned Leave", "Paternity Leave",'Total WD leaves'
]

df_leave_erp_summary = df_leave_erp_summary.drop(columns=[c for c in cols_to_drop if c in df_leave_erp_summary.columns], errors="ignore")

print(f"✅ LEAVE processed: {df_leave_erp_summary.shape}")
print("\n✅ EXEMPTED and LEAVE data processed!")

✅ LEAVE processed: (183, 4)

✅ EXEMPTED and LEAVE data processed!


In [25]:
df_leave_erp_summary.head()

,Emp Id,Name,Special Leave,Approved leaves (ERP)
0,GCU020107,Salema Khatun,0.0,19.0
1,GCU020083,Bedanta Bhattacharjee,0.0,4.0
2,GCU010032,Luit Chowdhury,0.0,3.0
3,GCU020109,Kriti Phukan,0.0,2.5
4,GCU030024,Kamrup Das,0.0,4.0


#### 5.1 Merge EXEMPTED and Calculate Final Values

In [26]:
# Merge EXEMPTED with actual data
df_fac_actual = df_fac_conso_ID.copy()
df_admin_actual = df_admin_conso_ID.copy()

df_fac_actual_exempted = pd.merge(df_fac_actual, df_exempted, how='left', on=['Emp Id'])
df_admin_actual_exempted = pd.merge(df_admin_actual, df_exempted, how='left', on=['Emp Id'])

df_fac_actual_exempted.fillna(0, inplace=True)
df_admin_actual_exempted.fillna(0, inplace=True)

#### 5.2 Calculate adjusted values (Half Days, Full Days, Late)

In [27]:
from utility_attendance import weighted_sum_and_replace_columns
for df in [df_fac_actual_exempted, df_admin_actual_exempted]:
    if not df.empty:
        # Convert to numeric and fill NaN with 0
        actual_am = pd.to_numeric(df['actual_AM_abs'], errors='coerce').fillna(0) if 'actual_AM_abs' in df.columns else 0
        actual_pm = pd.to_numeric(df['actual_PM_abs'], errors='coerce').fillna(0) if 'actual_PM_abs' in df.columns else 0
        actual_days_abs = pd.to_numeric(df['actual_days_abs'], errors='coerce').fillna(0) if 'actual_days_abs' in df.columns else 0
        actual_late = pd.to_numeric(df['actual_No_of_late'], errors='coerce').fillna(0) if 'actual_No_of_late' in df.columns else 0
        
        exempt_hd = pd.to_numeric(df['exempt_HD'], errors='coerce').fillna(0) if 'exempt_HD' in df.columns else 0
        exempt_fd = pd.to_numeric(df['exempt_FD'], errors='coerce').fillna(0) if 'exempt_FD' in df.columns else 0
        exempt_late = pd.to_numeric(df['exempt_late'], errors='coerce').fillna(0) if 'exempt_late' in df.columns else 0
        
        # Calculate half days (AM_abs + PM_abs - exempt_HD)
        df['Half Days'] = (actual_am + actual_pm - exempt_hd).clip(lower=0)
        df['Full Days'] = (actual_days_abs - exempt_fd).clip(lower=0)
        df['Late'] = (actual_late - exempt_late).clip(lower=0)

print("✅ EXEMPTED data merged")
print(f"   Faculty: {df_fac_actual_exempted.shape}")


✅ EXEMPTED data merged
   Faculty: (161, 24)


In [28]:
df_fac_actual_exempted.head()

,Emp Id,Name,Designation,Department,late_flags,early_flags,half_day_flags,actual_AM_abs,actual_PM_abs,actual_days_abs,...,actual_full_day,Working Days,Present,Absent,exempt_late,exempt_HD,exempt_FD,Half Days,Full Days,Late
0,GCU010013,Ms. Raisa,-,Administration,"10_16, 10_23",,,,,09_22,...,1,19,18.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,GCU010017,Ms. Ananya,-,Administration,,,,,,,...,0,19,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,GCU020001,Dr. Lipi,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,,...,0,19,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GCU020002,Dr. Swarnali,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,,...,0,19,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,,,,,,,...,0,19,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 5.3 Merge LEAVE Data and Calculate Observed Leaves

In [29]:
# Merge LEAVE with attendance
df_fac_report = pd.merge(df_fac_actual_exempted, df_leave_erp_summary, how='left', on='Emp Id', suffixes=('','_leave'))
df_admin_report = pd.merge(df_admin_actual_exempted, df_leave_erp_summary, how='left', on='Emp Id', suffixes=('','_leave'))

df_fac_report.fillna(0, inplace=True)
df_admin_report.fillna(0, inplace=True)

# Calculate Observed Leaves (weighted sum: Half Days=0.5, Full Days=1.0)
col_to_sum = ['Half Days', 'Full Days']
df_fac_report = weighted_sum_and_replace_columns(df_fac_report, col_to_sum, 'Observed Leaves', [0.5, 1.0])
df_admin_report = weighted_sum_and_replace_columns(df_admin_report, col_to_sum, 'Observed Leaves', [0.5, 1.0])

# Calculate Unauthorized leaves = Absent - Total WD leaves
if 'Absent' in df_fac_report.columns and 'Total WD leaves' in df_fac_report.columns:
    df_fac_report["Unauthorized leaves"] = (df_fac_report["Absent"] - df_fac_report["Total WD leaves"]).clip(lower=0)
if 'Absent' in df_admin_report.columns and 'Total WD leaves' in df_admin_report.columns:
    df_admin_report["Unauthorized leaves"] = (df_admin_report["Absent"] - df_admin_report["Total WD leaves"]).clip(lower=0)

print("✅ LEAVE data merged")
print(f"   Faculty report: {df_fac_report.shape}")
print(f"   Admin report: {df_admin_report.shape}")


✅ LEAVE data merged
   Faculty report: (161, 26)
   Admin report: (146, 26)


In [30]:
df_fac_report.columns

Index(['Emp Id', 'Name', 'Designation', 'Department', 'late_flags',
       'early_flags', 'half_day_flags', 'actual_AM_abs', 'actual_PM_abs',
       'actual_days_abs', 'No_of_AM_abs', 'No_of_PM_abs', 'actual_No_of_late',
       'actual_half_day', 'actual_full_day', 'Working Days', 'Present',
       'Absent', 'exempt_late', 'exempt_HD', 'exempt_FD', 'Late', 'Name_leave',
       'Special Leave', 'Approved leaves (ERP)', 'Observed Leaves'],
      dtype='object')

In [31]:
# Cell 10: Check GCU010013 Results
print("🔍 Checking results for GCU010013 (Raisa Choudhury)...")

if 'GCU010013' in df_fac_report['Emp Id'].values:
    test_result = df_fac_report[df_fac_report['Emp Id'] == 'GCU010013']
    print("\n✅ Found GCU010013:")
    print(f"   Working Days: {test_result['Working Days'].iloc[0]}")
    print(f"   Present: {test_result['Present'].iloc[0]}")
    print(f"   Absent: {test_result['Absent'].iloc[0]}")
    print(f"   Expected: Present=16.5, Absent=3.5")
    
    if abs(test_result['Present'].iloc[0] - 16.5) < 0.01 and abs(test_result['Absent'].iloc[0] - 3.5) < 0.01:
        print("\n✅ ✅ ✅ CORRECT! Results match expected values.")
    else:
        print("\n⚠️ Results don't match expected values")
        print(f"   Got: Present={test_result['Present'].iloc[0]}, Absent={test_result['Absent'].iloc[0]}")
else:
    print("⚠️ GCU010013 not found")

print(f"\n📊 Sample of all faculty:")
display(df_fac_report[['Emp Id', 'Name', 'Working Days', 'Present', 'Absent']].head(10))


🔍 Checking results for GCU010013 (Raisa Choudhury)...

✅ Found GCU010013:
   Working Days: 19
   Present: 18.0
   Absent: 1.0
   Expected: Present=16.5, Absent=3.5

⚠️ Results don't match expected values
   Got: Present=18.0, Absent=1.0

📊 Sample of all faculty:


,Emp Id,Name,Working Days,Present,Absent
0,GCU010013,Ms. Raisa,19,18.0,1.0
1,GCU010017,Ms. Ananya,19,19.0,0.0
2,GCU020001,Dr. Lipi,19,19.0,0.0
3,GCU020002,Dr. Swarnali,19,19.0,0.0
4,GCU020003,Dr. Kajal,19,19.0,0.0
5,GCU020006,Dr. Anindita,19,18.0,1.0
6,GCU020008,Dr. Nilanjana,19,19.0,0.0
7,GCU020009,Ms. Manjula,19,18.5,0.5
8,GCU020010,Mr. Adarsh,19,19.0,0.0
9,GCU020011,Dr. Madhumita,19,18.5,0.5


In [32]:
# 'Unauthorized leaves' df_admin_report
col_to_select = ['Emp Id', 'Name', 'Designation', 'Department','Working Days', 'Present', 'Absent','Late','Approved leaves (ERP)']
df_fac_report_final = df_fac_report[col_to_select].copy()
df_admin_report_final = df_admin_report[col_to_select].copy()

df_fac_report_final["Unauthorised leaves"] = (df_fac_report_final["Absent"] - df_fac_report_final["Approved leaves (ERP)"]).clip(lower=0)
df_admin_report_final["Unauthorised leaves"] = (df_admin_report_final["Absent"] - df_admin_report_final["Approved leaves (ERP)"]).clip(lower=0)
df_fac_report_final.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Unauthorised leaves
0,GCU010013,Ms. Raisa,-,Administration,19,18.0,1.0,2.0,1.0,0.0
1,GCU010017,Ms. Ananya,-,Administration,19,19.0,0.0,0.0,0.0,0.0
2,GCU020001,Dr. Lipi,Assistant Professor & HoD (I/c),School of Natural Science,19,19.0,0.0,0.0,0.0,0.0
3,GCU020002,Dr. Swarnali,Assistant Professor & HoD (I/c),School of Natural Science,19,19.0,0.0,0.0,0.0,0.0
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,19,19.0,0.0,0.0,0.0,0.0


In [33]:
df_admin_report_final.head(7)

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Unauthorised leaves
0,GCU010004,Kishore Kr.,-,Administration,19,9.0,10.0,9.0,4.0,6.0
1,GCU010005,Dwipen,Administration,Administration,19,18.5,0.5,15.0,0.5,0.0
2,GCU010011,Nripen Ch,-,Administration,19,18.5,0.5,0.0,0.0,0.5
3,GCU010012,Sourabh Mazinder,-,Administration,19,18.0,1.0,0.0,1.0,0.0
4,GCU010024,Mridul,-,Administration,19,19.0,0.0,1.0,0.0,0.0
5,GCU010025,Arup Kr.,Administration,Administration,19,18.5,0.5,0.0,0.5,0.0
6,GCU010026,Biraj Kr.,-,Administration,19,18.5,0.5,18.0,0.0,0.5


In [34]:
df_erp_name = df_emp[['Employee ID','Name']].copy()
df_erp_name.rename(columns = {'Employee ID': 'Emp Id'}, inplace=True)

In [35]:
df_erp_name.head()

,Emp Id,Name
0,SYSADMIN,System Administrator
1,APPLICANT_PORTAL_USER,APP_FIRST_NAME APP_MIDDLE_NAME APP_LAST_NAME
2,mis,MIS Admin
3,EMP4,Lokesh Yadav
4,EMP5,Birshu Sarma


In [36]:
df_erp_name[df_erp_name['Emp Id']=='GCU020003']

,Emp Id,Name
102,GCU020003,Kajal Dutta


In [37]:
df_fac_report_final[df_fac_report_final['Emp Id']=='GCU020003']

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Unauthorised leaves
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,19,19.0,0.0,0.0,0.0,0.0


In [38]:
df_admin_report_final[df_admin_report_final['Emp Id']=='GCU030042']

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Unauthorised leaves


In [39]:
df_erp_name.shape

(437, 2)

In [40]:
df_erp_name.Name.isnull().sum()

np.int64(5)

In [41]:
df_erp_name = df_erp_name.dropna(subset=['Name'])

In [42]:
df_erp_name = df_erp_name.drop_duplicates()
df_fac_report_final = df_fac_report_final.drop_duplicates()
df_admin_report_final = df_admin_report_final.drop_duplicates()
#df_erp_name.shape

In [43]:
df_erp_name['Emp Id'].duplicated().sum(), df_fac_report_final['Emp Id'].duplicated().sum(),df_admin_report_final['Emp Id'].duplicated().sum()

(np.int64(0), np.int64(0), np.int64(0))

In [44]:
df_fac_report_final.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Unauthorised leaves
0,GCU010013,Ms. Raisa,-,Administration,19,18.0,1.0,2.0,1.0,0.0
1,GCU010017,Ms. Ananya,-,Administration,19,19.0,0.0,0.0,0.0,0.0
2,GCU020001,Dr. Lipi,Assistant Professor & HoD (I/c),School of Natural Science,19,19.0,0.0,0.0,0.0,0.0
3,GCU020002,Dr. Swarnali,Assistant Professor & HoD (I/c),School of Natural Science,19,19.0,0.0,0.0,0.0,0.0
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,19,19.0,0.0,0.0,0.0,0.0


### 6. The final report

#### 6.1 Checking for missing Emp IDs

In [45]:
missing_ids = df_fac_report_final.loc[df_fac_report_final['Name'].isna(), 'Emp Id']
print("Missing Emp Ids (not found in ERP list):")
print(missing_ids.tolist())

Missing Emp Ids (not found in ERP list):
[]


#### 6.2 Cleaning before merging

In [46]:
# Normalise 'Emp Id' 
for df in [df_fac_report_final, df_admin_report_final, df_erp_name]:
    df['Emp Id'] = df['Emp Id'].astype(str).str.strip().str.upper()

In [47]:
# Delete 'Name' if it exist
for df in [df_fac_report_final, df_admin_report_final]:
    if 'Name' in df.columns:
        df.drop(columns='Name', inplace=True)

In [48]:
df_fac_report_final = df_fac_report_final.merge(df_erp_name, how='left', on='Emp Id')
df_admin_report_final = df_admin_report_final.merge(df_erp_name, how='left', on='Emp Id')

#### 6.3 Identify missing matches after both merges

In [49]:
missing_fac = df_fac_report_final.loc[df_fac_report_final['Name'].isna(), 'Emp Id']
missing_admin = df_admin_report_final.loc[df_admin_report_final['Name'].isna(), 'Emp Id']

print("Missing in Faculty:", len(missing_fac))
print("Missing in Admin:", len(missing_admin))

Missing in Faculty: 0
Missing in Admin: 4


In [50]:
# # Which missing only in Faculty
#set(missing_fac) - set(missing_admin)

#### 6.4 which IDs are not found:

In [51]:
missing_fac_ids = df_fac_report_final.loc[df_fac_report_final['Name'].isna(), 'Emp Id'].unique()
missing_admin_ids = df_admin_report_final.loc[df_admin_report_final['Name'].isna(), 'Emp Id'].unique()

print("Missing Faculty IDs:\n", missing_fac_ids)
print("Missing Admin IDs:\n", missing_admin_ids)

Missing Faculty IDs:
 []
Missing Admin IDs:
 ['GCU030172' 'GCU030192' 'GCU030193' 'GCU030194']


In [52]:
# pd.merge(df1, df2, on='id', how='inner')
#df_fac_report_final.drop(columns='Name',axis=1, inplace=True)
#df_admin_report_final.drop(columns='Name',axis=1, inplace=True)

#df_fac_report_final = pd.merge(df_fac_report_final,df_erp_name, how='left',on=['Emp Id'])
#df_admin_report_final = pd.merge(df_admin_report_final,df_erp_name, how='left',on=['Emp Id'])
                                

In [53]:
#df_admin_report_final.columns

In [54]:
corrected_order = ['Emp Id', 'Name', 'Designation', 'Department', 'Working Days', 'Present',
       'Absent', 'Late', 'Approved leaves (ERP)', 'Unauthorised leaves' ]
df_fac_report_final = df_fac_report_final[corrected_order]
df_admin_report_final = df_admin_report_final[corrected_order]

### Duplicates in merging

In [56]:
sheets_fac = ['Bio details','Bio Consolidated','ERP Leave','Exempted','Report']
dataframes_fac = [df_fac_detail, df_fac_conso, df_leave_erp_summary, df_exempted,df_fac_report_final]

with pd.ExcelWriter('./output/report_faculties_28_oct.xlsx', engine='xlsxwriter') as writer:
    for s, frame in zip(sheets_fac, dataframes_fac):
        frame.to_excel(writer, sheet_name=s, index=False)
print('✅ Faculties Excel report saved')

✅ Faculties Excel report saved


In [57]:
sheets_admin = ['Bio details','Bio Consolidated','ERP Leave','Exempted','Report']
dataframes_admin = [df_admin_detail, df_admin_conso, df_leave_erp_summary, df_exempted,df_admin_report_final]

with pd.ExcelWriter('./output/report_staffs_28_oct.xlsx', engine='xlsxwriter') as writer:
    for s, frame in zip(sheets_admin, dataframes_admin):
        frame.to_excel(writer, sheet_name=s, index=False)
print('✅ Staffs Excel report saved')

✅ Staffs Excel report saved
